<a href="https://colab.research.google.com/github/2022tumoresferas/EyCD/blob/main/2_EyC_Clasif_T_%20GD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
 <h2>Mentoría: Clasificación de Tumoresferas </h2>
</center>



<a name="exploratory_data_analysis"></a>
#### **Práctico de Análisis Exploratorio y Curación**

*FECHA LíMITE DE ENTREGA: 17/06*

En esta notebook, incorporamos parte de lo realizado en la 1era entrega ([carpeta de entrega de prácticos](https://drive.google.com/drive/folders/1Juit-kDMN3ZXgVXeC_myzx5jgYkjyJVI?usp=sharing)) por:
  
  * Alfredo Quevedo
  * Daniel Rubio 
  * Eugenia Bernaschini 
  * Guillermo A. Robiglio
  

  


Para asociar el Drive:

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
try:
  import os
  os.chdir("/content/drive/My Drive/mentoria/02_EyC")
  print("Acceso al directorio ---> OK")
  #os.chdir("/content/drive/My Drive/Datos_tumoresferas")

  # para acceder a la carpeta, en el siguiente link (Carpeta Guillermo):
  # https://drive.google.com/drive/folders/1Qsf63jIGYm57Ozsfl9biLSpgsqGiikBh?usp=sharing

except FileNotFoundError:
  print("problemas...acceso al directorio ---> NO OK")
  

Acceso al directorio ---> OK


Importamos las librerías necesarias:

In [3]:
from os import walk
import os.path
import csv
import pprint
import pdb


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context('talk') 

import warnings
warnings.filterwarnings("ignore")

Definimos algunas constantes.

In [4]:
# Se comenta la siguiente linea para dar lectura de los datos desde gdrive no local
# RUTA_A_ARCHIVOS_A_UNIR = "D:/Google-Drive-UNC/Diplodatos/Mentoria/data"

# Habiendo montado el drive de la mentoria, indicamos la ruta de acceso al repositorio de achivos a procesar
RUTA_A_ARCHIVOS_A_UNIR ="/content/drive/My Drive/mentoria/02_EyC" 
DELIMITADORES_CSV = [",", ";", ":"]
IDENTIFICADOR_DE_ARCHIVO = "modificado"

1) Para empezar a trabajar la tabla, primero hay que combinar los datasets, que fueron separados por día. 

Estos son los siguientes archivos a unir:

*   "fiji_datos_0al7mo_modificado_dia_0.csv"
*   "fiji_datos_0al7mo_modificado_dia_1.csv"
*   "fiji_datos_0al7mo_modificado_dia_2.csv"
*   "fiji_datos_0al7mo_modificado_dia_3.csv"
*   "fiji_datos_0al7mo_modificado_dia_4.csv"
*   "fiji_datos_0al7mo_modificado_dia_5.csv"
*   "fiji_datos_0al7mo_modificado_dia_6.csv"
*   "fiji_datos_0al7mo_modificado_dia_7a.csv"
*   "fiji_datos_0al7mo_modificado_dia_7b.csv"

2) Una vez armado el dataset modificado, vean si hay valores nulos, duplicados, etc, tratando de encontrar todas las inconsistencias en los datos. Decidir el orden en cuál ir arreglándolas.

3) Corregir las inconsistencias que van encontrando y sobre los datos faltantes (¡verificar si los hay!), determinar cuál método sería el mas adecuado para imputarlos, teniendo en cuenta lo que aprendieron sobre los datos en el práctico de *Análisis y Visualización*.

4) Una vez que recuperan el dataset original, repasando lo que les dieron en la materia de *Exploración y Curación de datos* y ya pensando en las transformaciones que puedan servirles y que les conviene realizar para la clasificación en el práctico de aprendizaje supervisado:

  * ¿Sobre cuáles columnas realizarían encoding y de qué tipo?.

  * Tenemos muchas columnas (¡aunque siempre puede haber mas!) por lo que es posible realizar algún método de reducción de dimensionalidad.

  Sumar tales transformaciones como columnas para sumar características al conjunto de datos y guardar el achivo modificado.

Si se traban mucho mucho en la limpieza de los datos, este paso pueden hacerlo con el dataset original que usaron en el práctico 1.

5) Realizar una documentación técnica de los procesos realizados. 






## Resolución.

**1)** Con el código presente en las siguientes celdas se obtiene un diccionario cuya **clave** es el **dia de toma de la muestra**, cada valor asociado es **otro diccionario** con la siguiente información descriptiva:
- Separador del CSV
- Cantidad de columnas
- Nombres de la columnas
- Nombre del archivo

Caben las siguientes aclaraciones:
- Con las constante `RUTA_A_ARCHIVOS_A_UNIR` se determina la ruta donde el código buscará los archivos que deben ser unidos un un mismo dataset.
- Como el código utiliza el método `walk` del módulo `os`, se provee al menos la ruta que se encuentra a un nivel por encima de la ubicación de los archivos. El código filtra aquellos archivos que contenga la palabra clave asignada a la constante `IDENTIFICADOR_DE_ARCHIVO`.
- A la constante `DELIMITADORES_CSV` se le provee una lista con todos los posibles delimitadores del los archivos CSV que se esperan encontrar.

In [5]:
def obtener_delimitador_csv(posibles_delimitadores: list, nombre_archivo_csv: str) -> str:
    '''Informa el delimitador de un archivo csv.

    Parámetros
    ----------
    posibles_delimitadores: lista de posibles delimitadores que se esperan encontrar.
    nombre_archivo_csv: cadena con el nombre del archivo csv.

    Salida
    ------
    delimitador: delimitador del archivo csv.
    '''
    for delimitador in posibles_delimitadores:
        with open(nombre_archivo_csv) as csvfile:
            datos = csv.reader(csvfile, delimiter=delimitador)
            columnas = datos.__next__()
            if len(columnas) > 1:
                return delimitador

def obtener_columnas(delimitador: str, nombre_archivo_csv: str) -> list:
    '''Provee las columnas del archivo csv cuyo nombre se ingresa como parámetro.
    
    Parámetros
    ----------
    delimitador: cadena de un caracter, delimitador de datos del archivo csv
    nombre_archivo_csv: cadena con el nombre del archivo csv.
    
    Salida
    ------
    columnas: lista con los nombres de las columnas del archivo csv
    '''
    with open(nombre_archivo_csv) as csvfile:
        datos = csv.reader(csvfile, delimiter=delimitador)
        columnas = datos.__next__()
        return columnas

def obtener_dia_muestreo(nombre_largo: str)-> int:
  '''Devuelve un entero con el ordinal del dia correspondiente al registro.

  Parámetros
  ----------
  nombre_largo: cadena de caracteres con el nombre del archivo a procesar

  Salida
  ------
  Un entero que representa el dia de la toma de las muestras
  '''

  cadena=nombre_largo
  cadena=cadena.replace("/content/drive/My Drive/mentoria/02_EyC/","")
  cadena=cadena.replace("fiji_datos_0al7mo_modificado_dia_","")
  cadena=cadena.replace(".csv","").replace("7a","7").replace("7b","8")
  return int(cadena)


In [6]:
info_archivo = {}
dict_info_archivos = {}

for raiz, _, nombres_de_archivo in walk(RUTA_A_ARCHIVOS_A_UNIR):
    #Breve tutorial sobre el metodo walk https://www.tutorialspoint.com/python/os_walk.htm

    for nombre_de_archivo in nombres_de_archivo:
    
        if IDENTIFICADOR_DE_ARCHIVO in nombre_de_archivo:
            # Breve tutorial sobre el metodo https://www.geeksforgeeks.org/python-os-path-join-method/
            nombre_completo_de_archivo = os.path.join(os.path.abspath(raiz), nombre_de_archivo)
    
            delimitador = obtener_delimitador_csv(DELIMITADORES_CSV, nombre_completo_de_archivo)
            columnas = obtener_columnas(delimitador, nombre_completo_de_archivo)
    
            # Diccionario por dia de muestreo con informacion del csv correspondiente
            info_archivo = {'nombre_archivo': nombre_completo_de_archivo,
                            'delimitador': delimitador,
                            'columnas': columnas,
                            'cant_columnas': len(columnas)}
            
            # Diccionario princial, con clave dia de toma de muestra 
            # obtenido del nombre del archivo csv
            dict_info_archivos[obtener_dia_muestreo(nombre_completo_de_archivo)]=info_archivo

pprint.pprint(dict_info_archivos)

{0: {'cant_columnas': 25,
     'columnas': ['',
                  'labels',
                  'Area',
                  'X',
                  'Y',
                  'XM',
                  'YM',
                  'Perim.',
                  'BX',
                  'BY',
                  'Width',
                  'Height',
                  'Circ.',
                  'Feret',
                  'FeretX',
                  'FeretY',
                  'FeretAngle',
                  'MinFeret',
                  'AR',
                  'Round',
                  'Solidity',
                  'Esferoide',
                  'dia',
                  'Diameter',
                  'n_diam'],
     'delimitador': ';',
     'nombre_archivo': '/content/drive/My '
                       'Drive/mentoria/02_EyC/fiji_datos_0al7mo_modificado_dia_0.csv'},
 1: {'cant_columnas': 24,
     'columnas': ['labels',
                  'Area',
                  'X',
                  'Y',
                  'XM'

Con el diccionario principal que contiene información de los archivos, se arman una lista "dataframes" con los respectivos datasets (lectura mediante). 

In [8]:
dataframes = []

for clave_dia in sorted(dict_info_archivos.keys()):
  info= dict_info_archivos[clave_dia]
  df = pd.read_csv(filepath_or_buffer = info['nombre_archivo'],
                     sep = info['delimitador'])
  dataframes.append(df)

A los fines de realizar una comparatativa entre las columnas del grupo de datos armamos un dataframe sintético llamado "df_cols" presentando en cada columna las respectivas columnas de cada set de datos por dia.  

In [9]:
df_cols = pd.DataFrame([])
i = 0

for df in dataframes:
    nombre_col = 'dia' + str(i)
    df_col = pd.DataFrame(df.columns, columns=[nombre_col])
    df_cols = pd.concat([df_cols, df_col], axis=1)
    i += 1

df_cols

,dia0,dia1,dia2,dia3,dia4,dia5,dia6,dia7,dia8
0,Unnamed: 0,labels,labels,Unnamed: 0,labels,labels,labels,Unnamed: 0,Unnamed: 0
1,labels,Area,Area,labels,Area,Area,Area,labels,labels
2,Area,X,X,Area,X,X,X,Area,Feret
3,X,Y,Y,X,Y,Y,Y,X,FeretX
4,Y,XM,XM,Y,XM,XM,XM,Y,FeretY
5,XM,YM,YM,XM,YM,YM,YM,XM,FeretAngle
6,YM,Perim.,Perim.,YM,Perim.,Perim.,Perim.,YM,MinFeret
7,Perim.,BX,BX,Perim.,BX,BX,BX,Perim.,AR
8,BX,BY,BY,BX,BY,BY,BY,BX,Round
9,BY,Width,Width,BY,Width,Width,Width,BY,Solidity


Se puede ver que algunos días tienen una columna adicional **Unnamed: 0** antes que labels. 

Se recorre la lista de datasets eliminando, de existir, la columna mencionada con el siguiente código.

In [10]:
for i in range(len(dataframes)):
  # Se agrega la clave ignore para que no emitar error 
  # al intentar borrar una columna que no existe
  dataframes[i].drop('Unnamed: 0', axis=1, inplace=True, errors = 'ignore')

Repetimos el dataframe sintético "df_cols".  

In [11]:
df_cols = pd.DataFrame([])
i = 0

for df in dataframes:
    nombre_col = 'dia' + str(i)
    df_col = pd.DataFrame(df.columns, columns=[nombre_col])
    df_cols = pd.concat([df_cols, df_col], axis=1)
    i += 1

df_cols

,dia0,dia1,dia2,dia3,dia4,dia5,dia6,dia7,dia8
0,labels,labels,labels,labels,labels,labels,labels,labels,labels
1,Area,Area,Area,Area,Area,Area,Area,Area,Feret
2,X,X,X,X,X,X,X,X,FeretX
3,Y,Y,Y,Y,Y,Y,Y,Y,FeretY
4,XM,XM,XM,XM,XM,XM,XM,XM,FeretAngle
5,YM,YM,YM,YM,YM,YM,YM,YM,MinFeret
6,Perim.,Perim.,Perim.,Perim.,Perim.,Perim.,Perim.,Perim.,AR
7,BX,BX,BX,BX,BX,BX,BX,BX,Round
8,BY,BY,BY,BY,BY,BY,BY,BY,Solidity
9,Width,Width,Width,Width,Width,Width,Width,Width,Esferoide


Se verifica mediante el siguiente código que existe coincidencia en los labels de los días 7a y 7b.

In [64]:
print(dataframes[7].shape, dataframes[8].shape)
sum(dataframes[7].labels == dataframes[8].labels)

(88, 12) (88, 13)


88

Asumiendo esta prueba como suficiente concluimos que estamos frente al mismo grupo de muestras (Dia 7) cuya información ha sido fragmentas en dos archivos complementarios.

Se procede a concatenar los mismos.


In [12]:
df_dia7a = dataframes[7]
df_dia7b = dataframes[8].drop('labels', axis=1)

df_dia7 = pd.concat([df_dia7a, df_dia7b], axis=1)
df_dia7.shape

(88, 24)

Eliminamos los datasets identificados como parciales (7, 8)

Luego agregamos el equivalente unificado en el dia 7. 

In [13]:
dataframes.pop() # Eliminación del día 7b
dataframes.pop() # Eliminación del día 7a
dataframes.append(df_dia7) # Completamiento de la lista de dataframes

Repetimos nuevamente el dataframe sintético "df_cols".

In [14]:
df_cols = pd.DataFrame([])
i = 0

for df in dataframes:
    nombre_col = 'dia' + str(i)
    df_col = pd.DataFrame(df.columns, columns=[nombre_col])
    df_cols = pd.concat([df_cols, df_col], axis=1)
    i += 1

df_cols

,dia0,dia1,dia2,dia3,dia4,dia5,dia6,dia7
0,labels,labels,labels,labels,labels,labels,labels,labels
1,Area,Area,Area,Area,Area,Area,Area,Area
2,X,X,X,X,X,X,X,X
3,Y,Y,Y,Y,Y,Y,Y,Y
4,XM,XM,XM,XM,XM,XM,XM,XM
5,YM,YM,YM,YM,YM,YM,YM,YM
6,Perim.,Perim.,Perim.,Perim.,Perim.,Perim.,Perim.,Perim.
7,BX,BX,BX,BX,BX,BX,BX,BX
8,BY,BY,BY,BY,BY,BY,BY,BY
9,Width,Width,Width,Width,Width,Width,Width,Width


La celda anterior pone en evidencia que todos los dataframes, desde el día 0 hasta el día 7 tienen la misma cantidad de columnas. Significa que están en condiciones para armar el dataframe modificado y continuar con el ejercicio 2.

In [15]:
df_modificado = pd.DataFrame([])

for df in dataframes:
    df_modificado = pd.concat([df_modificado, df], axis=0)

df_modificado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1171 entries, 0 to 87
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   labels      1171 non-null   object 
 1   Area        1171 non-null   float64
 2   X           1171 non-null   float64
 3   Y           1171 non-null   float64
 4   XM          1171 non-null   float64
 5   YM          1171 non-null   float64
 6   Perim.      1171 non-null   float64
 7   BX          1171 non-null   float64
 8   BY          1171 non-null   float64
 9   Width       1171 non-null   float64
 10  Height      1171 non-null   float64
 11  Circ.       1163 non-null   float64
 12  Feret       1171 non-null   float64
 13  FeretX      1171 non-null   int64  
 14  FeretY      1171 non-null   int64  
 15  FeretAngle  1171 non-null   float64
 16  MinFeret    1171 non-null   float64
 17  AR          1171 non-null   float64
 18  Round       1171 non-null   float64
 19  Solidity    1161 non-null   f

A los fines de aportar evidencias complementarias comparamos (**metodo info** de Pandas) por inspección el dataset reconstruido respecto de original (presuntamente fuera de nuestro alcance)

In [16]:
url = 'http://www.ccll.com.ar/fiji_datos_0al7mo_labels.csv'
df_original = pd.read_csv(url)
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018 entries, 0 to 1017
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   labels      1018 non-null   object 
 1   Area        1018 non-null   float64
 2   X           1018 non-null   float64
 3   Y           1018 non-null   float64
 4   XM          1018 non-null   float64
 5   YM          1018 non-null   float64
 6   Perim.      1018 non-null   float64
 7   BX          1018 non-null   float64
 8   BY          1018 non-null   float64
 9   Width       1018 non-null   float64
 10  Height      1018 non-null   float64
 11  Circ.       1018 non-null   float64
 12  Feret       1018 non-null   float64
 13  FeretX      1018 non-null   int64  
 14  FeretY      1018 non-null   int64  
 15  FeretAngle  1018 non-null   float64
 16  MinFeret    1018 non-null   float64
 17  AR          1018 non-null   float64
 18  Round       1018 non-null   float64
 19  Solidity    1018 non-null  

**2)** En base a la información del dataframe modificado, puede apreciarce que hay valores nulos.

Comparando con el dataframe original se puede apreciar también que existen valores adicionales que se sospecha podría tratarse de duplicados.

**3)** ...

**4)** ...

**5)** Las explicaciones dadas en esta notebook y los comentarios y docstrings que se hallan en los códigos de las celdas y en las funciones son la documentación que permite entender el trabajo realizado.

Se trató de automatizar el tratamiento de datos los más que se puedo en base al problema que se tiene y al tiempo disponible. Se espera que esto sirva de base para el tratamiento de problemas similares.

### *Opcionales:*

Si alguien se anima a cargar en alguna base de datos un dataset (original y/o el modificado), en SQL:

* Eliminar datos duplicados.
* Realizar algunas consultas sobre el dataset completo, por ej:
  * Cuántos datos hay por día, 
  * Cuántos esferoides "si" y "no" hay en total.
* En algún día en particular, consultar cuántos datos:
  * Son esferoides "si" y "no".
  * Tienen su diámetro entre 50$\mu$ m y 200$\mu$ m 
  